# 1. Grape Harvesting
- The Purpose of this Notebook Optimizes the amount of grapes which can be produced in a year, Depending on their harvest months

# Disregard

In [2]:
import pandas as pd
import plotly.graph_objects as go


In [3]:
# Define the harvesting times for both Red and White grapes

harvesting_times_Red = {
    "01": 100,  # January
    "02": 100,  # February
    "03": 100,  # March
    "04": 100,  # April
    "05": 100,  # May
    "06": 100,  # June
    "07": 100,  # July
    "08": 100,  # August
    "09": 1,  # September
    "10": 100,  # October
    "11": 100,  # November
    "12": 100   # December
}

harvesting_times_White = {
    "01": 100,  # January
    "02": 100,  # February
    "03": 100,  # March
    "04": 100,  # April
    "05": 100,  # May
    "06": 100,  # June
    "07": 100,  # July
    "08": 100,  # August
    "09": 1,  # September
    "10": 100,  # October
    "11": 100,  # November
    "12": 100   # December
}

In [4]:
# Create a DataFrame with the months, harvesting times for Red and White, and whether they're optimal
months = [str(i).zfill(2) for i in range(1, 13)]  # Months from 01 to 12

In [5]:
# Create the DataFrame
df = pd.DataFrame({
    "Month": months,
    "harvesting_times_Red": [harvesting_times_Red[month] for month in months],
    "optimal_months_Red": True,
    "harvesting_times_White": [harvesting_times_White[month] for month in months],
    "optimal_months_White": True
})

In [6]:
df["Sim_optimal_months_Red"] = df.apply(lambda x: x["optimal_months_Red"] if x["Month"] in ["09"] else False, axis=1)
df["Sim_optimal_months_White"] = df.apply(lambda x: x["optimal_months_White"] if x["Month"] in ["09"] else False, axis=1)

# df.rename(columns={"optimal_months_Red": "Sim_optimal_months_Red", "optimal_months_White": "Sim_optimal_months_White"}, inplace=True)

df = df[["Month", "harvesting_times_Red", "Sim_optimal_months_Red", 
         "harvesting_times_White", "Sim_optimal_months_White"]]


In [7]:
df

,Month,harvesting_times_Red,Sim_optimal_months_Red,harvesting_times_White,Sim_optimal_months_White
0,01,100,False,100,False
1,02,100,False,100,False
2,03,100,False,100,False
3,04,100,False,100,False
4,05,100,False,100,False
5,06,100,False,100,False
6,07,100,False,100,False
7,08,100,False,100,False
8,09,1,True,1,True
9,10,100,False,100,False


In [8]:
df.to_csv('../Output-Files/Grapes-Harvesting-Time.csv', index=False)

### Find the Long Run Average of number of wine batches produced in the next 15 years (15*12 Months)

- Red Grapes

In [9]:
# Simulating the dataframe for the next 10 years (120 Months)
total_months = 10 * 12
current_month = 0  # Starting from January
red_batches = 0
red_transport_ready = []

while current_month < total_months:
    month_index = current_month % 12  # To loop through the dataframe months
    # For Red Grapes
    if df.loc[month_index, "Sim_optimal_months_Red"]:
        red_batches += 1
        current_month += df.loc[month_index, "harvesting_times_Red"]  # Jumping to the next month after harvesting

        red_transport_ready.append(current_month)
    else:
        current_month += 1  # Moving to the next month if not optimal

print(red_batches)

10


- White Grapes

In [10]:
# Simulating the dataframe for the next 10 years (120 Months)
total_months = 10 * 12
current_month = 0  # Starting from January
white_batches = 0
white_transport_ready = []


while current_month < total_months:
    month_index = current_month % 12  # To loop through the dataframe months
    # For White Grapes, separate tracking to allow both to be harvested simultaneously
    if df.loc[month_index, "Sim_optimal_months_White"]:
        white_batches += 1
        current_month += df.loc[month_index, "harvesting_times_White"]  # Jumping to the next month after harvesting

        white_transport_ready.append(current_month)
    else:
        current_month += 1  # Moving to the next month if not optimal

print(white_batches)

10


- Visualisation of ready Grape batches Per Month

In [11]:
print(red_transport_ready)
print(white_transport_ready)

[9, 21, 33, 45, 57, 69, 81, 93, 105, 117]
[9, 21, 33, 45, 57, 69, 81, 93, 105, 117]


In [12]:
# Preparing data for the stacked bar chart, considering the simulation results for red and white grapes ready for transport
months_detailed = list(range(1, 121))  # Detailed months list for 10 years
red_ready_detailed = [1 if month in red_transport_ready else 0 for month in months_detailed]
white_ready_detailed = [1 if month in white_transport_ready else 0 for month in months_detailed]

# Adjusting for white grapes going beyond the 120th month
white_ready_detailed_adjusted = white_ready_detailed[:120] + [1 if 125 in white_transport_ready else 0]

# Plotting the bar graph to show when grapes are ready for transportation using the modified list
fig = go.Figure(data=[
    go.Bar(name='Red Grapes', x=months_detailed, y=red_ready_detailed, marker_color='red'),
    go.Bar(name='White Grapes', x=months_detailed, y=white_ready_detailed_adjusted, marker_color='green')
])

# Update layout for detailed view
fig.update_layout(
    barmode='stack',
    title="Monthly Grapes Ready for Transportation Over 10 Years",
    xaxis_title="Month (1-120 representing each month over 10 years)",
    yaxis_title="Batch Ready for Transportation (0 or 1)",
    yaxis=dict(tickmode='array', tickvals=[0, 1], ticktext=['Not Ready', 'Ready'])
)
fig.update_layout(template='plotly_dark')
fig.show()


### Analysis: About 4 batches of Red Grapes and 3 batches of White Grapes are harvested Every Year  

# 2. Transportation to Factory
- Transportation Analysis to factory is Insignificant since it takes only one day to and fro to the Factory

# 3. Factory Wine Profuction
- Find a way to Optimise the Number of Oak Barrels in the Factory
- Check if we need Additional Machines or not too 
- Check how to Optimise Inventory Level too

## Optimizing Number of Oak Barrels

In [13]:
# Red Wine 1: Pinot Noir Ler Cru

Crushing_and_Destemming = 1/30
Maceration = 1
Red_Wine_1_pre_oak_months = Crushing_and_Destemming + Maceration

Alcoholic_Fermentation = 1/2
Pressing = 1/30
Malolactic_Fermentation= 2
Red_Wine_1_oak_aging = 15
Red_Wine_1_oak_time = Alcoholic_Fermentation + Pressing + Malolactic_Fermentation + Red_Wine_1_oak_aging

print(Red_Wine_1_oak_time)

17.53333333333333


In [14]:
# Red Wine 2: Pinot Noir Grand Cru
Crushing_and_Destemming = 1/30
Maceration = 1
Red_Wine_2_pre_oak_months = Crushing_and_Destemming + Maceration

Alcoholic_Fermentation = 1/2
Pressing = 1/30
Malolactic_Fermentation= 2
Red_Wine_2_oak_aging = 24
Red_Wine_2_oak_time = Alcoholic_Fermentation + Pressing + Malolactic_Fermentation + Red_Wine_2_oak_aging

print(Red_Wine_2_oak_time)

26.53333333333333


In [15]:
# White Wine 1: Chardonnay Ler Cru
Crushing_and_Destemming = 1/30
Pressing = 1/30
White_Wine_1_pre_oak_months = Crushing_and_Destemming + Pressing

Alcoholic_Fermentation = 1/2
Malolactic_Fermentation= 3
White_Wine_1_oak_aging = 0
White_Wine_1_oak_time = Alcoholic_Fermentation + Malolactic_Fermentation + White_Wine_1_oak_aging

print(White_Wine_1_oak_time)

3.5


In [16]:
# White Wine 2: Chardonnay Grand Cru
Crushing_and_Destemming = 1/30
Pressing = 1/30
White_Wine_2_pre_oak_months = Crushing_and_Destemming + Pressing

Alcoholic_Fermentation = 1/2
Malolactic_Fermentation= 3
White_Wine_2_oak_aging = 12
White_Wine_2_oak_time = Alcoholic_Fermentation + Malolactic_Fermentation + White_Wine_2_oak_aging

print(White_Wine_2_oak_time)

15.5


In [17]:
# Initializing separate oak barrels usage tracking for each wine type
oak_barrels_red_1 = [0] * 120
oak_barrels_red_2 = [0] * 120
oak_barrels_white_1 = [0] * 120
oak_barrels_white_2 = [0] * 120

def update_oak_usage(transport_ready, oak_barrels_1, pre_oak_months, oak_aging_months): #Modify
    for start_month in transport_ready:
        oak_start = int(start_month + pre_oak_months) - 1  # Adjusting for 0-based indexing
        if oak_start < len(oak_barrels_1):  # Ensure within the 10-year period
            for i in range(oak_start, min(oak_start + int(oak_aging_months), len(oak_barrels_1))):
                oak_barrels_1[i] += 7  # Assuming each loop iteration represents one barrel used

def update_oak_usage_2(transport_ready, oak_barrels_1, pre_oak_months, oak_aging_months): #Modify
    for start_month in transport_ready:
        oak_start = int(start_month + pre_oak_months) - 1  # Adjusting for 0-based indexing
        if oak_start < len(oak_barrels_1):  # Ensure within the 10-year period
            for i in range(oak_start, min(oak_start + int(oak_aging_months), len(oak_barrels_1))):
                oak_barrels_1[i] += 3  # Assuming each loop iteration represents one barrel used

# Updating oak usage for all four types of wine
update_oak_usage(red_transport_ready, oak_barrels_red_1, Red_Wine_1_pre_oak_months, Red_Wine_1_oak_time)
update_oak_usage_2(red_transport_ready, oak_barrels_red_2, Red_Wine_2_pre_oak_months, Red_Wine_2_oak_time)
update_oak_usage(white_transport_ready, oak_barrels_white_1, White_Wine_1_pre_oak_months, White_Wine_1_oak_time)
update_oak_usage_2(white_transport_ready, oak_barrels_white_2, White_Wine_2_pre_oak_months, White_Wine_2_oak_time)

In [18]:
months = list(range(1, 121))

fig = go.Figure()
fig.add_trace(go.Scatter(x=months, y=oak_barrels_red_1, mode='lines', name='Red Wine 1', stackgroup='one', marker_color='darkred'))
fig.add_trace(go.Scatter(x=months, y=oak_barrels_red_2, mode='lines', name='Red Wine 2', stackgroup='one', marker_color='red'))
fig.add_trace(go.Scatter(x=months, y=oak_barrels_white_1, mode='lines', name='White Wine 1', stackgroup='one', marker_color='lightgreen'))
fig.add_trace(go.Scatter(x=months, y=oak_barrels_white_2, mode='lines', name='White Wine 2', stackgroup='one', marker_color='green'))

fig.update_layout(
    title="Stacked Monthly Oak Barrel Usage for Each Wine Type Over 10 Years",
    xaxis_title="Month",
    yaxis_title="Cumulative Number of Oak Barrels",
    hovermode="x unified"
)

fig.show()

### Analysis on Optimizing Oak Barrels in the Factory
- The Long Term Average of Number of Oak Barrels: 56
  - Red Wine (18.5 Months)
  - Red Wine (27.5 Months)
  - White Wine (3.5 Months)
  - White Wine (15.5 Months)

## Optimizing Inventory Levels in the Factory Warehouse 

# 4. Transportation from Factory to Local Warehouse
- Check How many Planes are required for Transportation (Most Likely only one)
- One Truck for Each Country to Transport wine from the Destination Airport to Local Warehose 

# 5. Transportation from Local Warehouse to Retail Store 
- Most likely only one too
- One Truck to Transport Wine from Local Warehouse to Retail Store (Same Truck Used for Aeroplane Transportation)
  - Prioritise Aeroplane Transportation Over anything Else